In [12]:

from langchain.agents import initialize_agent, Tool, tool
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from bootstrap.toolchain import get_tools

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools=get_tools(),
    llm=ChatOpenAI(temperature=0.7, model_name='gpt-4', ), 
    agent="conversational-react-description", 
    verbose=True,
    memory=memory,
    )

/Users/alexandernaka/Library/Caches/pypoetry/virtualenvs/bootstrap-TdbWQQRJ-py3.10/lib/python3.10/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/alexandernaka/Library/Caches/pypoetry/virtualenvs/bootstrap-TdbWQQRJ-py3.10/lib/python3.10/site-packages/langchain/llms/openai.py:608: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [13]:
agent.run('what functions and classes are in this repo?')



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Get repo definitions summary
Action Input: None
Observation: [{'qualname': 'bootstrap.auth.set_environment_vars', 'type': 'function', 'signature': "set_environment_vars(method, 'dotenv')", 'docstring': None}, {'qualname': 'bootstrap.interface_setup.create_human_message_template', 'type': 'function', 'signature': 'create_human_message_template()', 'docstring': None}, {'qualname': 'bootstrap.interface_setup.create_chat_prompt', 'type': 'function', 'signature': 'create_chat_prompt(human_message)', 'docstring': None}, {'qualname': 'bootstrap.interface_setup.create_conversation_chain', 'type': 'function', 'signature': 'create_conversation_chain(chat_prompt)', 'docstring': None}, {'qualname': 'bootstrap.dummy_module.dummy_function', 'type': 'function', 'signature': 'dummy_function(a, b)', 'docstring': None}, {'qualname': 'bootstrap.dummy_module.DummyClass', 'type': 'class', 'signature': 'class DummyClass', 

'In this repo, there are several functions and classes:\n\nFunctions:\n1. `bootstrap.auth.set_environment_vars`\n2. `bootstrap.interface_setup.create_human_message_template`\n3. `bootstrap.interface_setup.create_chat_prompt`\n4. `bootstrap.interface_setup.create_conversation_chain`\n5. `bootstrap.dummy_module.dummy_function`\n6. `bootstrap.introspection.get_full_definition`\n7. `bootstrap.introspection.extract_function_info`\n8. `bootstrap.introspection.extract_method_info`\n9. `bootstrap.introspection.extract_class_info`\n10. `bootstrap.introspection.process_nodes`\n11. `bootstrap.introspection.extract_functions_and_classes`\n12. `bootstrap.introspection.get_current_repo_functions_and_classes`\n13. `bootstrap.introspection.get_current_repo_definitions_summary`\n14. `bootstrap.introspection._get_parent_module_source_code`\n15. `bootstrap.introspection._get_source_code`\n16. `bootstrap.introspection.get_source_code`\n17. `bootstrap.introspection._edit_source_code`\n18. `bootstrap.intros

In [14]:
agent.run('give me a list of the functions and classes that currently lack a docstring')



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Get repo definitions summary
Action Input: None
Observation: [{'qualname': 'bootstrap.auth.set_environment_vars', 'type': 'function', 'signature': "set_environment_vars(method, 'dotenv')", 'docstring': None}, {'qualname': 'bootstrap.interface_setup.create_human_message_template', 'type': 'function', 'signature': 'create_human_message_template()', 'docstring': None}, {'qualname': 'bootstrap.interface_setup.create_chat_prompt', 'type': 'function', 'signature': 'create_chat_prompt(human_message)', 'docstring': None}, {'qualname': 'bootstrap.interface_setup.create_conversation_chain', 'type': 'function', 'signature': 'create_conversation_chain(chat_prompt)', 'docstring': None}, {'qualname': 'bootstrap.dummy_module.dummy_function', 'type': 'function', 'signature': 'dummy_function(a, b)', 'docstring': None}, {'qualname': 'bootstrap.dummy_module.DummyClass', 'type': 'class', 'signature': 'class DummyClass', 

'The following functions and classes currently lack a docstring:\n\nFunctions:\n1. `bootstrap.auth.set_environment_vars`\n2. `bootstrap.interface_setup.create_human_message_template`\n3. `bootstrap.interface_setup.create_chat_prompt`\n4. `bootstrap.interface_setup.create_conversation_chain`\n5. `bootstrap.dummy_module.dummy_function`\n6. `bootstrap.repo2docstore.create_faiss_index`\n7. `bootstrap.repo2docstore.get_github_docs`\n8. `bootstrap.toolchain.get_tools`\n9. `bootstrap.codebase_index.save_codebase_llama_index`\n10. `bootstrap.codebase_index.load_codebase_llama_index`\n11. `tests.test_introspection.test_get_source_code`\n12. `tests.test_introspection.test_edit_source_code`\n13. `tests.test_interface_setup.test_load_environment_variables`\n14. `tests.test_interface_setup.test_create_chat_prompt`\n\nClasses:\n1. `bootstrap.dummy_module.DummyClass`'

In [15]:
agent.run('write a numbered list of the functions and classes that currently lack a docstring. then write a docstring for the first 3')



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Here is a numbered list of the functions and classes that currently lack a docstring:

Functions:
1. `bootstrap.auth.set_environment_vars`
2. `bootstrap.interface_setup.create_human_message_template`
3. `bootstrap.interface_setup.create_chat_prompt`
4. `bootstrap.interface_setup.create_conversation_chain`
5. `bootstrap.dummy_module.dummy_function`
6. `bootstrap.repo2docstore.create_faiss_index`
7. `bootstrap.repo2docstore.get_github_docs`
8. `bootstrap.toolchain.get_tools`
9. `bootstrap.codebase_index.save_codebase_llama_index`
10. `bootstrap.codebase_index.load_codebase_llama_index`
11. `tests.test_introspection.test_get_source_code`
12. `tests.test_introspection.test_edit_source_code`
13. `tests.test_interface_setup.test_load_environment_variables`
14. `tests.test_interface_setup.test_create_chat_prompt`

Classes:
1. `bootstrap.dummy_module.DummyClass`

Now, I will write docstrings for the first three fu

'Here is a numbered list of the functions and classes that currently lack a docstring:\n\nFunctions:\n1. `bootstrap.auth.set_environment_vars`\n2. `bootstrap.interface_setup.create_human_message_template`\n3. `bootstrap.interface_setup.create_chat_prompt`\n4. `bootstrap.interface_setup.create_conversation_chain`\n5. `bootstrap.dummy_module.dummy_function`\n6. `bootstrap.repo2docstore.create_faiss_index`\n7. `bootstrap.repo2docstore.get_github_docs`\n8. `bootstrap.toolchain.get_tools`\n9. `bootstrap.codebase_index.save_codebase_llama_index`\n10. `bootstrap.codebase_index.load_codebase_llama_index`\n11. `tests.test_introspection.test_get_source_code`\n12. `tests.test_introspection.test_edit_source_code`\n13. `tests.test_interface_setup.test_load_environment_variables`\n14. `tests.test_interface_setup.test_create_chat_prompt`\n\nClasses:\n1. `bootstrap.dummy_module.DummyClass`\n\nNow, I will write docstrings for the first three functions:\n\n1. `bootstrap.auth.set_environment_vars`\n```py

In [ ]:
agent.run('write a numbered list of the functions and classes that currently lack a docstring, then write a docstring for the first 3')



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Here is a numbered list of the functions and classes that currently lack a docstring:

Functions:
1. `bootstrap.auth.set_environment_vars`
2. `bootstrap.interface_setup.create_human_message_template`
3. `bootstrap.interface_setup.create_chat_prompt`
4. `bootstrap.interface_setup.create_conversation_chain`
5. `bootstrap.dummy_module.dummy_function`
6. `bootstrap.repo2docstore.create_faiss_index`
7. `bootstrap.repo2docstore.get_github_docs`
8. `bootstrap.toolchain.get_tools`
9. `bootstrap.codebase_index.save_codebase_llama_index`
10. `bootstrap.codebase_index.load_codebase_llama_index`
11. `tests.test_introspection.test_get_source_code`
12. `tests.test_introspection.test_edit_source_code`
13. `tests.test_interface_setup.test_load_environment_variables`
14. `tests.test_interface_setup.test_create_chat_prompt`

Classes:
1. `bootstrap.dummy_module.DummyClass`

Now, I will write docstrings for the first three fu

'Here is a numbered list of the functions and classes that currently lack a docstring:\n\nFunctions:\n1. `bootstrap.auth.set_environment_vars`\n2. `bootstrap.interface_setup.create_human_message_template`\n3. `bootstrap.interface_setup.create_chat_prompt`\n4. `bootstrap.interface_setup.create_conversation_chain`\n5. `bootstrap.dummy_module.dummy_function`\n6. `bootstrap.repo2docstore.create_faiss_index`\n7. `bootstrap.repo2docstore.get_github_docs`\n8. `bootstrap.toolchain.get_tools`\n9. `bootstrap.codebase_index.save_codebase_llama_index`\n10. `bootstrap.codebase_index.load_codebase_llama_index`\n11. `tests.test_introspection.test_get_source_code`\n12. `tests.test_introspection.test_edit_source_code`\n13. `tests.test_interface_setup.test_load_environment_variables`\n14. `tests.test_interface_setup.test_create_chat_prompt`\n\nClasses:\n1. `bootstrap.dummy_module.DummyClass`\n\nNow, I will write docstrings for the first three functions:\n\n1. `bootstrap.auth.set_environment_vars`\n```py